In [52]:
import numpy as np
from sklearn.datasets import load_boston
from sklearn.cross_validation import KFold

import revrand.basis_functions as bf
from revrand.validation import smse, msll
from revrand.regression import learn, predict
from revrand.btypes import Parameter, Positive

import revrand.legacygp as gp
import revrand.legacygp.kernels as kern


In [53]:
# Load the data
boston = load_boston()
X = boston.data
y = boston.target - boston.target.mean()

N, D = X.shape
lenscale = 1000

In [54]:
# Construct basis functions
base = bf.RandomRBF(Xdim=D, nbases=50, lenscale_init=Parameter(lenscale*np.ones(D), Positive())) \
    + bf.RandomRBF(Xdim=D, nbases=50, lenscale_init=Parameter(lenscale, Positive()))
    

In [55]:
# Construct a GP kernel to compare to
def kdef(h, k):
    return (h(1e-5, 10., 0.5) * k(kern.gaussian, h(1e-5, 1e5, 1.)) +
            k(kern.lognoise, h(-4, 1, 0)))


In [56]:
# Cross val
folds = 5
av_smse, av_msll = 0., 0.
av_smse_gp, av_msll_gp = 0., 0

for i, (tr_ind, ts_ind) in enumerate(KFold(len(y), n_folds=folds, shuffle=True)):
    
    # Training
    params = learn(X[tr_ind], y[tr_ind], base)
     
    hyper_params = gp.learn(X[tr_ind], y[tr_ind], kdef, verbose=False, ftol=1e-15)
    regressor = gp.condition(X[tr_ind], y[tr_ind], kdef, hyper_params)
    
    # Prediction
    Ey, Vf, Vy = predict(X[ts_ind], base, *params)
    
    query = gp.query(regressor, X[ts_ind])
    Ey_gp = gp.mean(query)
    Vy_gp = gp.variance(query, noise=True)

    
    # Validation
    f_smse, f_msll = smse(y[ts_ind], Ey), msll(y[ts_ind], Ey, Vy, y[tr_ind])
    f_smse_gp, f_msll_gp = smse(y[ts_ind], Ey_gp), msll(y[ts_ind], Ey_gp, Vy_gp, y[tr_ind])

    av_smse += f_smse
    av_msll += f_msll
    av_smse_gp += f_smse_gp
    av_msll_gp += f_msll_gp
    
    print("Fold: {},\n\trevrand: SMSE = {}, MSLL = {}"
          "\n\tGP: SMSE = {}, MSLL = {}"
          .format(i, f_smse, f_msll, f_smse_gp, f_smse_gp))
    
av_smse /= folds
av_msll /= folds
av_smse_gp /= folds
av_msll_gp /= folds

      fun: 2232.1988450530002
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
      jac: array([ -90.04779713,    0.        , -244.77531042])
  message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 24
      nit: 2
   status: 0
  success: True
        x: (10.0, 1.0000000017384991e-05, 1.0)
Fold: 0,
	revrand: SMSE = 0.14592126041401096, MSLL = -0.9885358286104551
	GP: SMSE = 1.001903054407219, MSLL = 1.001903054407219
      fun: 1910.9313905185788
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
      jac: array([ -65.39462447,    0.        , -227.90029561])
  message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 68
      nit: 8
   status: 0
  success: True
        x: (10.0, 8.832284762628452, 1.0)
Fold: 1,
	revrand: SMSE = 0.3263870305004232, MSLL = -0.5093013267088291
	GP: SMSE = 0.4522816620168146, MSLL = 0.4522816620168146
      fun: 1878.0812793312948
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
      jac: array([ -61.941

In [57]:
# Print results
print("Final:\n\trevrand: SMSE = {}, MSLL = {}"
      "\n\tGP: SMSE = {}, MSLL = {}"
      .format(av_smse, av_msll, av_smse_gp, av_msll_gp))


Final:
	revrand: SMSE = 0.18777701033553756, MSLL = -0.8753221924347155
	GP: SMSE = 0.680596997974403, MSLL = 1.151122070110981
